<a href="https://colab.research.google.com/github/LanxyL/Capstone3---Recommendation-System/blob/main/Notebook/Recommendation_System_on_Amazon_home_kitchen_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error

In [4]:
#Read Kitchen&Dining Dataset from last notebook
KitchenDining = pd.read_csv('drive/My Drive/Recommendation_System/KitchenDining80000.csv')

In [5]:
# transform the ratings frame into a ratings matrix
ratings_mtx_df = KitchenDining.pivot_table(values='overall',
                                           index='reviewerID',
                                           columns='asin')
ratings_mtx_df.head(3)

asin,0912696591,B00000JGRP,B00000JGRQ,B00000JGRT,B00002N602,B00004OCIP,B00004OCIQ,B00004OCIU,B00004OCIX,B00004OCIZ,B00004OCJ6,B00004OCJ9,B00004OCJJ,B00004OCJK,B00004OCJN,B00004OCJO,B00004OCJQ,B00004OCJW,B00004OCK0,B00004OCK3,B00004OCKO,B00004OCKR,B00004OCKT,B00004OCL2,B00004OCL8,B00004OCL9,B00004OCLA,B00004OCLK,B00004OCLW,B00004OCM4,B00004OCMB,B00004OCMM,B00004OCMS,B00004OCMX,B00004OCMY,B00004OCNE,B00004OCNJ,B00004OCNK,B00004OCNL,B00004OCNQ,...,B00JW02L9W,B00JWXSPAS,B00JWY1PPE,B00JWYX7YQ,B00JWZCE08,B00JZKZPMO,B00JZL11V2,B00JZXYG9O,B00K11U0Y4,B00K4S1L1K,B00K4V4EWA,B00K6BCV7S,B00K7ILPYU,B00K7P0D9G,B00K8LHA3G,B00KAF5DLQ,B00KAJ3OY0,B00KAK443Y,B00KB7KNRM,B00KBQ1OHQ,B00KBUGQJS,B00KCFDB4U,B00KD83BMS,B00KDV5RZY,B00KFS40N0,B00KI2RQHU,B00KIVD1PC,B00KKYSBUW,B00KM9W2PU,B00KNPVL8W,B00KQCJ0CG,B00KQXDIG4,B00KSRQN0Q,B00KSRSZC0,B00KSRV3UQ,B00KSRXUC0,B00KW1ZDCM,B00KWKQ2MS,B00L8FQZ3M,B00LB18EKK
reviewerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A00473363TJ8YSZ3YAGG9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A00593903MCE5GXTAGGNL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A037357115LCU92V2R0YR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(KitchenDining, test_size=0.2)

In [11]:
print(train.shape)
print(test.shape)
print(KitchenDining.shape)

(65558, 24)
(16390, 24)
(81948, 24)


In [14]:
def evaluate(estimate_f):
  """ RMSE-based predictive performance evaluation with pandas. """
  ids_to_estimate = zip(test.reviewerID, test.asin) 
  estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
  real = test.overall.values
  return np.sqrt(mean_squared_error(real, estimated))

In [16]:
#Predict all reviews as 5 star and see how accurate it would be
def simple_estimate_function(user_id, item_id): return 5

In [18]:
print('RMSE for simple estimate function: %s' % evaluate(simple_estimate_function))

RMSE for simple estimate function: 1.1773746660820885


Since we already know that most of the people rate 5 on the review. Let us set this estimation function as a threshold. A better model shall have a much smaller RMSE.

In [28]:
test.overall.isnull().sum()

0

In [30]:
def content_mean(user_id, item_id):
  """ Simple content-filtering based on mean ratings. """
  if user_id in train.reviewerID.to_list():
    user_condition = train.reviewerID == user_id
    return train.loc[user_condition, 'overall'].mean()
  else:
    return 5
print('RMSE for content mean estimation: %s' % evaluate(content_mean))

RMSE for content mean estimation: 1.0365250483549582


In [32]:
def collaborative_mean(user_id, item_id):
  """ Simple collaborative-filtering based on mean ratings. """
  if item_id in train.asin.to_list():
    item_condition = train.asin == item_id
    return train.loc[item_condition, 'overall'].mean()
  else:
    return 5
print('RMSE for collaborative mean estimation: %s' % evaluate(collaborative_mean))

RMSE for collaborative mean estimation: 0.9851181675150297
